In [0]:
# Databricks - installation des libs Python
%pip install azure-storage-blob requests tqdm python-dotenv

# Cellule 1 : Imports
import requests
from datetime import datetime
from tqdm import tqdm
import os

# Cellule 2 : Configuration Azure Data Lake Gen2
STORAGE_ACCOUNT_NAME = "datalakecertifimpe"
STORAGE_ACCOUNT_KEY = "xxxxxxxxxxxxxx" # Remplace par ta clé
CONTAINER_NAME = "data-gouv"

# URL du fichier Parquet SIRENE
PARQUET_URL = "https://object.files.data.gouv.fr/data-pipeline-open/siren/stock/StockEtablissement_utf8.parquet" 

print("✅ Configuration chargée")
print(f"📦 Compte de stockage : {STORAGE_ACCOUNT_NAME}")
print(f"📂 Conteneur : {CONTAINER_NAME}")

# Cellule 3 : Configuration Spark pour Azure Data Lake Gen2
spark.conf.set(
    f"fs.azure.account.key.{STORAGE_ACCOUNT_NAME}.dfs.core.windows.net",
    STORAGE_ACCOUNT_KEY
)

print("✅ Configuration Spark pour ADLS Gen2 effectuée")


Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
✅ Configuration chargée
📦 Compte de stockage : datalakecertifimpe
📂 Conteneur : data-gouv
✅ Configuration Spark pour ADLS Gen2 effectuée


In [0]:

# Cellule 4 : Fonction de téléchargement avec barre de progression
def download_parquet_with_progress(url, local_path):
    """
    Télécharge un fichier avec barre de progression
    """
    try:
        print(f"📥 Démarrage du téléchargement...")
        print(f"🔗 URL : {url}")
        
        # Requête HEAD pour obtenir la taille du fichier
        response = requests.head(url, allow_redirects=True, timeout=30)
        file_size = int(response.headers.get('content-length', 0))
        
        print(f"📦 Taille du fichier : {file_size / (1024**3):.2f} GB")
        
        # Téléchargement avec streaming et barre de progression
        response = requests.get(url, stream=True, timeout=300)
        response.raise_for_status()
        
        # Créer le dossier si nécessaire
        os.makedirs(os.path.dirname(local_path), exist_ok=True)
        
        # Téléchargement avec barre de progression
        with open(local_path, 'wb') as file:
            with tqdm(total=file_size, unit='B', unit_scale=True, desc="Téléchargement") as pbar:
                for chunk in response.iter_content(chunk_size=8192):
                    if chunk:
                        file.write(chunk)
                        pbar.update(len(chunk))
        
        print(f"✅ Téléchargement terminé : {local_path}")
        return True
        
    except Exception as e:
        print(f"❌ Erreur lors du téléchargement : {str(e)}")
        return False


In [0]:
# Cellule 5 : Téléchargement du fichier Parquet
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
# Chemin pour Python (avec /dbfs/)
local_parquet_path = f"/dbfs/tmp/sirene_stock_etablissement_{timestamp}.parquet"
# Chemin pour Spark (avec dbfs:/ sans /dbfs/)
spark_parquet_path = f"dbfs:/tmp/sirene_stock_etablissement_{timestamp}.parquet"

success = download_parquet_with_progress(PARQUET_URL, local_parquet_path)

if success:
    # Vérifier la taille du fichier téléchargé
    file_size_mb = os.path.getsize(local_parquet_path) / (1024**2)
    print(f"📊 Taille locale : {file_size_mb:.2f} MB")
    print(f"📍 Chemin Python : {local_parquet_path}")
    print(f"📍 Chemin Spark : {spark_parquet_path}")


📥 Démarrage du téléchargement...
🔗 URL : https://object.files.data.gouv.fr/data-pipeline-open/siren/stock/StockEtablissement_utf8.parquet
📦 Taille du fichier : 1.94 GB


Téléchargeme

*** WARNING: max output size exceeded, skipping output. ***

Téléchargement: 100%|██████████| 2.09G/2.09G [03:45<00:00, 9.25MB/s]

✅ Téléchargement terminé : /dbfs/tmp/sirene_stock_etablissement_20251114_183648.parquet
📊 Taille locale : 1990.99 MB
📍 Chemin Python : /dbfs/tmp/sirene_stock_etablissement_20251114_183648.parquet
📍 Chemin Spark : dbfs:/tmp/sirene_stock_etablissement_20251114_183648.parquet


In [0]:
# Cellule 6 : Upload vers Azure Data Lake Gen2
if success:
    try:
        print(f"\n{'='*60}")
        print(f"☁️  Upload vers Azure Data Lake Gen2...")
        
        # Chemin de destination dans ADLS Gen2
        adls_path = f"abfss://{CONTAINER_NAME}@{STORAGE_ACCOUNT_NAME}.dfs.core.windows.net/sirene/raw/sirene_stock_etablissement_{timestamp}.parquet"
        
        # Utiliser le chemin dbfs:/ pour Spark (sans /dbfs au début)
        dbfs_path = f"dbfs:/tmp/sirene_stock_etablissement_{timestamp}.parquet"
        
        # Lecture du fichier local en tant que DataFrame Spark
        df = spark.read.parquet(dbfs_path)
        
        print(f"📊 Nombre de lignes : {df.count():,}")
        print(f"📋 Schéma du fichier :")
        df.printSchema()
        
        # Écriture dans ADLS Gen2
        print(f"💾 Écriture dans ADLS Gen2...")
        df.write.mode("overwrite").parquet(adls_path)
        
        print(f"✅ Fichier uploadé avec succès dans ADLS Gen2")
        print(f"📍 Chemin : {adls_path}")
        
        # Nettoyage du fichier temporaire
        os.remove(local_parquet_path)
        print(f"🧹 Fichier temporaire supprimé")
        
    except Exception as e:
        print(f"❌ Erreur lors de l'upload : {str(e)}")


☁️  Upload vers Azure Data Lake Gen2...
📊 Nombre de lignes : 42,151,993
📋 Schéma du fichier :
root
 |-- siren: string (nullable = true)
 |-- nic: long (nullable = true)
 |-- siret: string (nullable = true)
 |-- statutDiffusionEtablissement: string (nullable = true)
 |-- dateCreationEtablissement: date (nullable = true)
 |-- trancheEffectifsEtablissement: string (nullable = true)
 |-- anneeEffectifsEtablissement: long (nullable = true)
 |-- activitePrincipaleRegistreMetiersEtablissement: string (nullable = true)
 |-- dateDernierTraitementEtablissement: timestamp_ntz (nullable = true)
 |-- etablissementSiege: boolean (nullable = true)
 |-- nombrePeriodesEtablissement: long (nullable = true)
 |-- complementAdresseEtablissement: string (nullable = true)
 |-- numeroVoieEtablissement: string (nullable = true)
 |-- indiceRepetitionEtablissement: string (nullable = true)
 |-- dernierNumeroVoieEtablissement: string (nullable = true)
 |-- indiceRepetitionDernierNumeroVoieEtablissement: string (

In [0]:
# Cellule 7 : Vérification et aperçu des données
try:
    print(f"\n{'='*60}")
    print(f"🔍 Vérification des données dans ADLS Gen2...")
    
    # Relire depuis ADLS Gen2
    df_adls = spark.read.parquet(adls_path)
    
    print(f"✅ Lecture réussie depuis ADLS Gen2")
    print(f"📊 Nombre de lignes : {df_adls.count():,}")
    print(f"📋 Nombre de colonnes : {len(df_adls.columns)}")
    
    print(f"\n🔎 Aperçu des 5 premières lignes :")
    df_adls.show(5, truncate=False)
    
    print(f"\n📈 Statistiques de base :")
    df_adls.describe().show()
    
except Exception as e:
    print(f"❌ Erreur lors de la vérification : {str(e)}")


🔍 Vérification des données dans ADLS Gen2...
✅ Lecture réussie depuis ADLS Gen2
📊 Nombre de lignes : 42,151,993
📋 Nombre de colonnes : 53

🔎 Aperçu des 5 premières lignes :
+---------+---+--------------+----------------------------+-------------------------+-----------------------------+---------------------------+----------------------------------------------+----------------------------------+------------------+---------------------------+------------------------------+-----------------------+-----------------------------+------------------------------+----------------------------------------------+---------------------+----------------------------+-----------------------+---------------------------+-----------------------------------+---------------------------------+------------------------+----------------------+-------------------------+-----------------------------+--------------------------------+-------------------------------+--------------------------------------+----------